In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
folder_path = '/kaggle/input/cic-nids2017'
all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.csv')]


In [3]:
all_files

['Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
 'Tuesday-WorkingHours.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
 'Monday-WorkingHours.pcap_ISCX.csv',
 'Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'Wednesday-workingHours.pcap_ISCX.csv']

In [4]:
specific_csv = 'Monday-WorkingHours.pcap_ISCX.csv'
if specific_csv in all_files:
    all_files.remove(specific_csv)

In [5]:
import pandas as pd
list_of_dfs = [pd.read_csv(os.path.join(folder_path, file)) for file in all_files]
combined_df = pd.concat(list_of_dfs, ignore_index=True)

In [6]:
combined_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300820,53,32215,4,2,112,152,28,28,28.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2300821,53,324,2,2,84,362,42,42,42.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2300822,58030,82,2,1,31,6,31,0,15.500000,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2300823,53,1048635,6,2,192,256,32,32,32.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
combined_df[' Label'].value_counts()

 Label
BENIGN                        1743179
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [16]:
df = combined_df.sample(500000)
df.columns = df.columns.str.strip()
labels_to_keep = ['BENIGN', 'DoS Hulk', 'PortScan', 'DDoS']
rows_to_drop = df[~df['Label'].isin(labels_to_keep)].index
df = df.drop(rows_to_drop)

In [18]:
def outliers_capping(feature,feature_name,dataset):
    # Identify 25th & 75th quartiles
#     q25, q75 = np.percentile(feature, 25), np.percentile(feature, 75)
#     feat_iqr = q75 - q25

#     feat_cut_off = feat_iqr * 1.5
#     feat_lower, feat_upper = q25 - feat_cut_off, q75 + feat_cut_off

#     outliers = [x for x in feature if (x < feat_lower and x != -9999999) or x > feat_upper]
#     print(feature_name + ' outliers: {}'.format(len(outliers)))

#     upper_idxes = dataset[(dataset[feature_name] > feat_upper) | (dataset[feature_name] == float('inf'))].index
#     lower_idxes = dataset[(dataset[feature_name] < feat_lower)].index

#     upper_outliers = dataset.loc[upper_idxes].copy()
#     lower_outliers = dataset.loc[lower_idxes].copy()
    idxes = dataset[(dataset[feature_name] == float('inf')) | (dataset[feature_name] == -float('inf'))].index
    max_val = max(dataset.drop(idxes)[feature_name])
#     outliers = dataset.loc[idxes].copy()
    dataset[feature_name][idxes] = max_val
#     # Cap outliers
#     dataset[feature_name][upper_idxes] = feat_upper
#     dataset[feature_name][lower_idxes] = feat_lower

#     # Create DataFrame of outliers for this feature
#     outliers_df = pd.concat([upper_outliers, lower_outliers])
#     outliers_df['outlier_column'] = feature_name
#     print('-' * 65)
#     return outliers_df
for col in df.drop(columns = 'Label'):
    outliers_capping(df[col],str(col),df)


In [19]:
X = df.drop(columns = 'Label')
y = df['Label']
# # Add these columns to make the feature map a square
# X['0col1'] = 1
# X['0col2'] = 1
# X['0col3'] = 1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y = le.fit_transform(y)
labels = y
features = X.values

In [20]:
pd.Series(y).value_counts()

0    378862
2     50399
3     34442
1     27745
Name: count, dtype: int64

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491448 entries, 1524617 to 1250169
Data columns (total 78 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Destination Port             491448 non-null  int64  
 1   Flow Duration                491448 non-null  int64  
 2   Total Fwd Packets            491448 non-null  int64  
 3   Total Backward Packets       491448 non-null  int64  
 4   Total Length of Fwd Packets  491448 non-null  int64  
 5   Total Length of Bwd Packets  491448 non-null  int64  
 6   Fwd Packet Length Max        491448 non-null  int64  
 7   Fwd Packet Length Min        491448 non-null  int64  
 8   Fwd Packet Length Mean       491448 non-null  float64
 9   Fwd Packet Length Std        491448 non-null  float64
 10  Bwd Packet Length Max        491448 non-null  int64  
 11  Bwd Packet Length Min        491448 non-null  int64  
 12  Bwd Packet Length Mean       491448 non-null  float64
 1

In [52]:
sorted(X['Flow Bytes/s'], reverse = True)

[2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 2070000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 1040000000.0,
 690000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.0,
 617000000.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.2,
                                                 stratify = y,
                                                 random_state = 42)

params = {
                'eta' : [0.1,0.01,0.001],  # Learning rate
                'eval_metric': ['logloss'],
                'max_depth' : [3,6,9],
                'lambda' : [1,1.5,2],
                'alpha' : [0,0.5,1]
            }
rnd_model_ = RandomizedSearchCV(XGBClassifier(random_state = 42, error_score = 'raise'), params, n_iter=4, scoring = 'f1')
rnd_model_.fit(X_train, y_train)
best_model=rnd_model_.best_estimator_

y_p_train = best_model.predict(X_train)
y_p_test = best_model.predict(X_test)

from sklearn.metrics import classification_report

print('For train data:')
print(classification_report(y_p_train, y_train))

print('For test data:')
print(classification_report(y_p_test, y_test))

In [10]:
# features = tf.convert_to_tensor(features)

NameError: name 'tf' is not defined

In [48]:
# import tensorflow_transform as tft

# tft.scale_by_min_max(
#     features,
#     0.0,
#     1.0,
#     elementwise = False,
#     name = None
# ) 

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [22]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)
features

array([[ 0.08272985, -0.45283569, -0.0098715 , ..., -0.11009894,
        -0.37512885, -0.35788601],
       [-0.31908489, -0.45283592, -0.01129036, ..., -0.11009894,
        -0.37512885, -0.35788601],
       [-0.40825692, -0.44359093, -0.00277721, ..., -0.11009894,
        -0.37512885, -0.35788601],
       ...,
       [-0.40825692, -0.44655687, -0.00277721, ..., -0.11009894,
        -0.37512885, -0.35788601],
       [-0.43045034, -0.45195334, -0.01129036, ..., -0.11009894,
        -0.37512885, -0.35788601],
       [-0.43045034, -0.45074808, -0.0098715 , ..., -0.11009894,
        -0.37512885, -0.35788601]])

In [24]:
import tensorflow as tf
import numpy as np
n_samples = features.shape[0]
padded_features = np.zeros((n_samples, 81))
padded_features[:, :features.shape[1]] = features
# normalized_features = tf.keras.utils.normalize(padded_features, axis=1)
X = padded_features.reshape((-1, 9, 9, 1))

In [25]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, categories='auto')
one_hot_encoded = encoder.fit_transform(labels.reshape(-1,1))
one_hot_encoded

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [11]:
X.shape

(225745, 9, 9, 1)

In [53]:
one_hot_encoded.shape

(225745, 2)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import initializers

custom_initializer = initializers.RandomUniform(minval=-1, maxval=1)
#tf.keras.initializers.HeNormal()

model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(9, 9, 1), kernel_initializer = custom_initializer))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', kernel_initializer = custom_initializer))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

model.fit(X, one_hot_encoded, epochs=100, batch_size=64)  # Adjust epochs and batch_size as neededfrom IPython.display import Audio

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 8, 8, 32)          160       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 4, 4, 32)         128       
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 3, 16)          2064      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 1, 16)         0         
 2D)                                                             
                                                      

KeyboardInterrupt: 

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(9, 9, 1), kernel_initializer = tf.keras.initializers.HeNormal()))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', kernel_initializer = tf.keras.initializers.HeNormal()))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', kernel_initializer = tf.keras.initializers.HeNormal()))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))
model.add(Flatten())

# model.add(Dense(81, activation='relu'),input_shape=(81,))
model.add(Dense(64, activation='sigmoid', kernel_initializer = tf.keras.initializers.GlorotUniform()))
model.add(Dense(32, activation='sigmoid', kernel_initializer = tf.keras.initializers.GlorotUniform()))
model.add(Dense(4, activation='softmax', kernel_initializer = tf.keras.initializers.GlorotUniform()))

# model = Sequential([
#     Dense(units=81, activation='relu', input_dim=81),   # assuming your input data shape is (batch_size, 81)
#     Dense(units=64, activation='relu'),
#     Dense(units=32, activation='relu'),
#     Dense(2, activation='softmax')
# ])
# labels = np.asarray(labels).astype('float32').reshape((-1,1))
initial_learning_rate = 0.000001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.66,
    staircase=True)
model.compile(optimizer=tf.keras.optimizers.Adam(lr = lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, one_hot_encoded, epochs=100, batch_size=8, validation_split = 0.2)  # Adjust epochs and batch_size as neededfrom IPython.display import Audio
# audio = Audio(filename="/kaggle/input/train-alarm/mixkit-classic-winner-alarm-1997 (online-audio-converter.com).mp3")
# display(audio)

Epoch 1/100
 3014/49145 [>.............................] - ETA: 2:22 - loss: nan - accuracy: 0.7716

KeyboardInterrupt: 

In [25]:
a = 2
a

2